In [1]:
import numpy as np
import pandas as pd

## Reading in dbNSFP filtered for just entries that contain clinvar

In [2]:
db = pd.read_csv('../data/dbNSFP3.2.clinvar.genes.txt',sep='\t')
print(db.shape)

/Users/d/work/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (0,7,8,9,10,66,67,68,69,70,71,72,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,106,107) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(38465, 183)


## Clinvar clinical significance encoding from here:
https://www.ncbi.nlm.nih.gov/clinvar/docs/clinsig/

## These are all the unique pathogenicity scores 

In [3]:
db['clinvar_clnsig'].unique()

array(['5', '3', '2', '5|5|5|5|5|5|5', '5|5', '4', '3|2', '5|2', '2|2',
       '5|4|4', '5|5|5|5', '5|3', '5|5|5|5|5', '5|2|2|2|2|2', '5|4|5',
       '4|5', '3|3|3', '5|5|5', '5|2|2|2|2', '5|5|5|4', '5|4', '2|2|2',
       '2|3|2|3', '3|3', '2|2|2|2', '5|5|4|4', '4|4|4|4|4|4', '2|3',
       '5|5|2', '5|2|2', '4|4', '-5', '4|4|4', '5|5|5|5|5|5', '5|5|5|5|2',
       '5|5|5|5|5|5|5|5', '5|5|5|5|5|5|5|5|5', '4|4|5', '4|5|4', '5|5|4',
       '3|5', '5|5|2|2', '4|4|4|4|4|4|4', '3|4', '5|5|3', '5|3|4',
       '2|2|2|3|2|2|2', '5|3|5', '5|5|5|2|3|3', '2|2|3', '3|5|3|4',
       '5|3|3', '5|2|5', '3|2|3', '6', '6|6', '6|6|6|6|6', '3|2|2',
       '4|5|5', '4|3|5|4', '2|4', '2|5', '5|4|5|5', '6|2|2', '4|3|3',
       '3|2|3|2|2', '4|4|4|4|4', '4|5|4|5', '4|3', '5|5|3|3|3', '3|3|3|3',
       '2|3|3|2', '3|3|2', '2|3|2', '5|3|2|2', '5|5|5|5|2|2|2|2|2|2|2',
       '5|5|5|2|5|2|2|2|2', '5|4|4|4|4', '3|2|2|2', '3|3|3|2', '5|2|3',
       '2|3|2|2', '2|2|2|2|2', '2|2|3|2|2', '3|3|2|3', '3|2|2|3|2',
       

In [4]:
db[db['clinvar_clnsig'].str.contains('-')].shape

(12, 183)

# How I plan to clean up the clinvar significance values!
### Multiple variants
- Since we can only interpret 2,3,4,5 I will remove variants with any other scores. 
- I will pick the most common values if there is one.
- If there is not I will pick the most deleterious one.

### Negative values

There are several values with negative values for the significance, doing some digging, the negatives mean the
the score is for the ref allele. 

- Drop these values for this analysis since it is only 12 variants

In [5]:
from statistics import mode

In [6]:
clean_sigs = []

for c in db['clinvar_clnsig']:
    
    if '-' in c:
        clean_sigs.append(np.nan)
        continue
        
    sigs = list(map(int,c.split('|')))
    clean_sig = [s for s in sigs if s in [2,3,4,5] ]
    
    if len(clean_sig) == 0:
        clean_sigs.append(np.nan)
        continue
    
    try:
        sig = mode(clean_sig)
    except:
        sig = max(clean_sig)

    clean_sigs.append(sig)
db['clean_clinvar_clnsig'] = clean_sigs 

In [7]:
path_dct = {2:'benign',
            3:'likely benign',
            4:'likely pathogenic',
            5:'pathogenic'}

db['clinvar_pathogenic'] = [path_dct[d] if d in path_dct.keys() else np.nan for d in db['clean_clinvar_clnsig']]

In [8]:
print(db.shape)
print('Filtering for just benign or pathogenic variants')
db = db[~db['clinvar_pathogenic'].isnull()]
db = db.reset_index(drop=True)
print(db.shape)

(38465, 185)
Filtering for just benign or pathogenic variants
(38405, 185)


In [9]:
db.to_csv('../data/dbNSFP3.2.clinvar_clean.txt',
          sep='\t')

## Create table for just proteins with glycosylation sites

In [10]:
p_and_aa = pd.read_csv('../data/protein_and_aa.txt',
                       sep='\t',
                       header=None)

In [11]:
ps = set(p_and_aa[0].to_list())
ix_good = []
db_prots = db['Uniprot_acc_Polyphen2']
for i in db_prots.index:
    for p in db_prots[i].split(';'):
        if p in ps:
            ix_good.append(i)

In [12]:
print('Just clinvar variants in proteins with glycosylation sites')
db_f = db.loc[ix_good]
print(db_f.shape)
db_f = db_f.drop_duplicates()
print(db_f.shape)

Just clinvar variants in proteins with glycosylation sites
(5795, 185)
(5739, 185)


## Add column to indicate whether they are near a glycosylation site

In [13]:
j_and_aa_dict = {p:list(p_and_aa[p_and_aa[0] == p][1]) for p in p_and_aa[0].unique()}

In [14]:
match_index = []
for i in db_f.index:
    one_var = db_f.loc[i]
    proteins = one_var['Uniprot_acc_Polyphen2']
    aaposs = one_var['aapos']
    for prot in proteins.split(';'):
        if prot in j_and_aa_dict.keys():
            aas = j_and_aa_dict[prot]
            for aa in map(int,aaposs.split(';')):
                if aa in aas:
                    match_index.append(i)

In [15]:
db_f['glycosite_proximal'] = [True if i in match_index else False for i in db_f.index]

In [16]:
db_f.to_csv('../data/clinvar_glycoproteins.txt',sep='\t')